In [1]:
import os

os.chdir("../..")
os.getcwd()

'c:\\Mohit\\Imperial\\fyp-hearts'

In [2]:
import utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from configs.wavelet_configs import *
import glob
from experiments.ecg_experiment import ECG_Experiment

In [3]:
outputfolder = os.getcwd()+'/output/'
datafolder = os.getcwd()+'/datasets/PTB-XL/'

model = conf_wavelet_single_rf
models = [model]

def generate_summary_table(selection=None, exps=None, folder='output/'):
    if exps is None:
        exps = ['exp0', 'exp1', 'exp1.1', 'exp1.1.1', 'exp2', 'exp3']
    metric1 = 'macro_auc'
    
    # getmodels
    models = {}
    for i,exp in enumerate(exps):
        if selection is None:
            exp_models = [m.split('/')[-1] for m in glob.glob(f'{folder}{exp}/models/*')]
        else:
            exp_models = selection
        if i==0:
            models = set(exp_models)
        else:
            models = models.union(set(exp_models))
    
    results_dic = {'Method': []}
    for exp in exps:
        results_dic[f'{exp}_AUC'] = []
    
    for model in models:
        results_dic['Method'].append(model)
        
        for e in exps:
            try:
                me_res = pd.read_csv(folder+str(e)+'/models/'+str(model)+'/results/te_results.csv', index_col=0)
                
                mean1 = me_res.loc['point'][metric1]
                unc1 = max(me_res.loc['upper'][metric1]-me_res.loc['point'][metric1], me_res.loc['point'][metric1]-me_res.loc['lower'][metric1])
                
                results_dic[e+'_AUC'].append("%.3f(%.2d)" %(np.round(mean1,3), int(unc1*1000)))
            except FileNotFoundError:
                results_dic[e+'_AUC'].append("---")
    
    df = pd.DataFrame(results_dic)
    df_index = df[df.Method.isin(['naive', 'ensemble'])]
    df_rest = df[~df.Method.isin(['naive', 'ensemble'])]
    df = pd.concat([df_rest, df_index])
    df.to_csv(folder+'results_ptbxl.csv')
    
    print(df)
    return df

generate_summary_table(selection=[model['modelname']])
print("----")

             Method   exp0_AUC   exp1_AUC exp1.1_AUC exp1.1.1_AUC   exp2_AUC  \
0  Wavelet+RF+lead1  0.727(18)  0.740(17)  0.741(16)    0.760(12)  0.632(26)   

    exp3_AUC  
0  0.804(22)  
----


In [4]:
name = 'exp0'
task = 'all'
e = ECG_Experiment(name, task, datafolder, outputfolder, models)
e.prepare()

[{'modelname': 'Wavelet+RF+lead1', 'modeltype': 'WAVELET', 'modelparams': {'regularizer_C': 0.001, 'classifier': 'RF'}}]
c:\Mohit\Imperial\fyp-hearts/output/exp0/data/
prepared


In [6]:
data, raw_labels = utils.load_dataset(datafolder, 100)


In [9]:
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)

In [15]:
test_labels = labels[labels.strat_fold==10]
test_data = data[labels.strat_fold==10]

In [22]:
test_labels_male = test_labels[test_labels.sex==0]
test_data_male = test_data[test_labels.sex==0]

test_labels_female = test_labels[test_labels.sex==1]
test_data_female = test_data[test_labels.sex==1]

In [26]:
test_data_male.shape, test_data_female.shape

((1132, 1000, 12), (1066, 1000, 12))

In [80]:
experiments = [
        # ('exp0', 'all'),
        # ('exp1', 'diagnostic'),
        # ('exp1.1', 'subdiagnostic'),
        # ('exp1.1.1', 'superdiagnostic'),
        # ('exp2', 'form'),
        ('exp3', 'rhythm')
    ]

models = [
    conf_wavelet_standard_rf,
        conf_wavelet_standard_nn,
        conf_wavelet_single_nn,
        conf_wavelet_single_rf,
        conf_wavelet_single_xgb,
        conf_wavelet_standard_xgb,
]

for name, task in experiments:
    print(name)
    data, raw_labels = utils.load_dataset(datafolder, 100)
    labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
    labels = labels[labels.rhythm_len > 0]
    test_labels = labels[labels.strat_fold<9]
    
        
    test_labels_male = np.where(test_labels.sex==0)

    test_labels_female = np.where(test_labels.sex==1)
    
    for model in models:
        print(model)
        modelname = model['modelname']
        y_path = os.getcwd()+"\\output\\"+name+"\\data\y_train.npy"
        y_pred_path = os.getcwd()+"\\output\\"+name+"\\models\\"+modelname+"\\y_train_pred.npy"
        y = np.load(y_path, allow_pickle=True)
        y_pred = np.load(y_pred_path, allow_pickle=True)
        
        y_male = y[test_labels_male]
        y_male = y_male[:,[i for i in range(y_male.shape[1]) if len(np.unique(y_male[:,i])) == 2]]
        y_female = y[test_labels_female]
        y_female = y_female[:,[i for i in range(y_female.shape[1]) if len(np.unique(y_female[:,i])) == 2]]
        
        
        y_pred_male = y_pred[test_labels_male]
        y_pred_male = y_pred_male[:,[i for i in range(y_male.shape[1]) if len(np.unique(y_male[:,i])) == 2]]
        y_pred_female = y_pred[test_labels_female]
        y_pred_female = y_pred_female[:,[i for i in range(y_female.shape[1]) if len(np.unique(y_female[:,i])) == 2]]
        auc_male = utils.evaluate_experiment(y_male, y_pred_male, thresholds=None)
        auc_female = utils.evaluate_experiment(y_female, y_pred_female, thresholds=None)
        print(auc_male, auc_female)

exp3
{'modelname': 'Wavelet+RF', 'modeltype': 'WAVELET', 'modelparams': {'regularizer_C': 0.001, 'classifier': 'RF'}}
   macro_auc
0        1.0    macro_auc
0        1.0
{'modelname': 'Wavelet+NN', 'modeltype': 'WAVELET', 'modelparams': {'regularizer_C': 0.001, 'classifier': 'NN'}}
   macro_auc
0    0.92623    macro_auc
0   0.931088
{'modelname': 'Wavelet+NN+lead1', 'modeltype': 'WAVELET', 'modelparams': {'regularizer_C': 0.001, 'classifier': 'NN'}}
   macro_auc
0   0.848188    macro_auc
0    0.86611
{'modelname': 'Wavelet+RF+lead1', 'modeltype': 'WAVELET', 'modelparams': {'regularizer_C': 0.001, 'classifier': 'RF'}}
   macro_auc
0        1.0    macro_auc
0        1.0
{'modelname': 'Wavelet+XGB+lead1', 'modeltype': 'WAVELET', 'modelparams': {'tree': 'hist', 'classifier': 'XGB'}}
   macro_auc
0   0.994035    macro_auc
0   0.994518
{'modelname': 'Wavelet+XGB', 'modeltype': 'WAVELET', 'modelparams': {'tree': 'hist', 'classifier': 'XGB'}}
   macro_auc
0   0.999955    macro_auc
0   0.999868

In [86]:
auc_male.values[0][0]

0.9999549549549549

In [52]:
test_labels_np = np.array(test_labels)
a = np.where(test_labels.sex==0)
len(a[0])
test_labels_np[list(a)]

C:\Users\agarw\AppData\Local\Temp\ipykernel_21216\1477625422.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  test_labels_np[list(a)]


array([[18792.0, 55.0, 0, ..., 'records500/00000/00009_hr', 2,
        list(['SR', 'NORM'])],
       [17076.0, 40.0, 0, ..., 'records500/00000/00038_hr', 2,
        list(['SR', 'NORM'])],
       [19501.0, 60.0, 0, ..., 'records500/00000/00040_hr', 2,
        list(['SR', 'NORM'])],
       ...,
       [17746.0, 85.0, 0, ..., 'records500/21000/21796_hr', 5,
        list(['AFIB', 'ASMI', 'IRBBB', 'IMI', 'ABQRS'])],
       [20789.0, 67.0, 0, ..., 'records500/21000/21812_hr', 5,
        list(['SVARR', '1AVB', 'LAFB', 'ABQRS', 'CRBBB'])],
       [9843.0, 54.0, 0, ..., 'records500/21000/21819_hr', 3,
        list(['LAFB', 'SR', 'IRBBB'])]], dtype=object)

'c:\\Mohit\\Imperial\\fyp-hearts'